# Data Preparation

Code authored by: Shaw Talebi

### imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

from openai import OpenAI
from dotenv import load_dotenv
import os

In [2]:
# setup api client
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

### functions

In [3]:
def compute_openai_embedding(text, column_name: str, model_name: str = 'text-embedding-3-small'):
    """
    Compute an embedding for a given text using OpenAI's embeddings API.
    
    Args:
        text (str or list-like): The text to compute an embedding for
        column_name (str): The name of the column to compute an embedding for
        model_name (str): OpenAI model to use (default: 'text-embedding-3-small')
    
    Returns:
        embeddings: numpy array of shape (N, 1536)
    """
    response = client.embeddings.create(input = f"{column_name}: " + text, model=model_name)
    embedding_list = [embedding_object.embedding for embedding_object in response.data]
    return np.array(embedding_list)

### load data

In [4]:
df_students = pd.read_csv('data/students.csv')
df_students = df_students.set_index('Id')
print(df_students.shape)

(119, 2)


In [5]:
# drop missing course join responses
df_students = df_students[df_students['course_join_question'] != 'No response provided']
print(df_students.shape)

(94, 2)


In [6]:
# drop missing job titles
df_students = df_students[df_students['job_title'] != 'Unknown job title']
print(df_students.shape)

(83, 2)


#### join reason

Embed join reasons and reduce dimensionality with PCA

In [7]:
# embed join reasons
embeddings = compute_openai_embedding(df_students['course_join_question'], "Course Join Reason Response: ")

In [8]:
# Compute PCA
n_components = 5
pca = PCA(n_components=n_components)
principal_components = pca.fit_transform(embeddings)

# Scale each column to [-1, 1] range
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled_components = scaler.fit_transform(principal_components)

In [9]:
# add scaled components to df
for i in range(n_components):
    df_students[f'join_reason_pc{i+1}'] = scaled_components[:, i]

In [10]:
df_students.head()

,job_title,course_join_question,join_reason_pc1,join_reason_pc2,join_reason_pc3,join_reason_pc4,join_reason_pc5
Id,,,,,,,
2,Product Manager,Understand Gen AI usecases,0.836463,0.772963,-0.173617,0.098713,0.425706
3,Founder,I would like to get practical experience in bu...,0.702372,-0.305023,0.463640,0.086446,-0.398564
6,Data Analyst/Economist,I would like to start to have a grasp of how I...,1.000000,0.277557,0.252502,0.478106,0.712033
9,VP Finance,Two things: (1) Build AI applications to strea...,0.502678,-0.213529,0.099288,1.000000,0.511309
10,Head of Product,Uplevel my knowledge on AI and ideally be able...,0.829856,0.028958,0.313570,-0.178677,0.623492


#### job titles

Categorize job titles using embedding model

In [11]:
# define job categories
job_category_dict = {
    "IC": [
        "engineer", "analyst", "scientist", "developer", "designer", "specialist",
        "consultant", "associate", "technician", "coordinator", "assistant",
        "architect", "writer", "editor", "researcher", "contributor", "operator"
    ],
    "Manager": [
        "manager", "lead", "supervisor", "head of", "team lead", "co-lead",
        "project lead", "project manager", "product manager", "scrum master"
    ],
    "Leader": [
        "director", "vp", "vice president", "chief", "cfo", "ceo", "coo", "cto", "cmo",
        "executive", "head", "senior director", "managing director", "president", "partner"
    ],
    "Entrepreneur": [
        "founder", "co-founder", "owner", "entrepreneur", "self-employed", "independent consultant",
        "business owner", "startup", "freelancer", "solo founder"
    ],
    "Student": [
        "student", "intern", "graduate student", "phd", "undergraduate", "bachelor", "master", 
        "mba", "candidate", "doctoral", "research assistant", "teaching assistant"
    ]
}

job_category_ser = pd.Series({k: f"{k} ({', '.join(v)})" for k, v in job_category_dict.items()})
job_category_list = job_category_ser.index.to_list()

In [12]:
# compute embeddings for job titles and categories
title_embeddings = compute_openai_embedding(df_students['job_title'], "Job Title: ")
category_embeddings = compute_openai_embedding(job_category_ser, "Job Category: ")

In [13]:
# match title to categories based on similarity

# Normalize embeddings for cosine similarity calculation
title_embeddings_norm = title_embeddings / np.linalg.norm(title_embeddings, axis=1, keepdims=True)
category_embeddings_norm = category_embeddings / np.linalg.norm(category_embeddings, axis=1, keepdims=True)

# Compute cosine similarity using dot product
similarities = np.dot(title_embeddings_norm, category_embeddings_norm.T)

# Get the best matching category for each title
best_category_indices = np.argmax(similarities, axis=1)
df_students['job_category'] = [job_category_list[idx] for idx in best_category_indices]

In [14]:
# encode job_category using 1-hot encoding
for category in job_category_list:
    df_students[f'job_category_{category}'] = (df_students['job_category'] == category).astype(int)

### join with review data

In [15]:
# load review data
df_reviews = pd.read_csv('data/reviews.csv')

In [16]:
# join with student data
df = df_students.join(df_reviews.set_index('id'), how='left')

In [17]:
# create boolean for 5 star review
df['5_star_review'] = df['rating'] == 10.0

### save data

In [19]:
df.to_csv('data/students_reviews.csv')